In [1]:
import os
os.chdir('../')

## Imports

In [2]:
import pandas as pd
import datetime as dt

from risk_suite.economics import EconomicsCalculator

## Data Collect

#### Score Imobiliária

In [3]:
scores_4KST = (
    pd.read_csv("data/scores.csv")
    [["dt_ativacao", "id_contrato", "faixa_score"]]
    .rename(columns={
        "id_contrato": "contract_id",
        "faixa_score": "rating_4KST"
    })
)
scores_4KST.dt_ativacao = pd.to_datetime(scores_4KST["dt_ativacao"]).dt.strftime("%Y-%m") 

/var/folders/nc/c031_2vx4zgf4ldqyn14mvvm0000gq/T/ipykernel_2550/193310261.py:2: DtypeWarning: Columns (5) have mixed types. Specify dtype option on import or set low_memory=False.
  pd.read_csv("data/scores.csv")


In [164]:
df_new_imobs = pd.read_csv("notebooks/outputs/df_new_imobs.csv")
df_bins = pd.read_csv("notebooks/outputs/df_bins_2.csv")

# Alinhando classificções com cohort de origem dos dados
# df_bins.dt_ativacao = (pd.to_datetime(df_bins["dt_ativacao"]) - pd.DateOffset(months=6)).dt.strftime("%Y-%m") 

df_new_imobs["optimal_segments"] = "4-novas"
df_bins = pd.concat([df_bins, df_new_imobs], ignore_index=True)
df_bins.dt_ativacao = (pd.to_datetime(df_bins["dt_ativacao"])).dt.strftime("%Y-%m") 

df_bins.head()

,dt_ativacao,id_imobiliaria,predictions,target,old_segments,optimal_bins,optimal_segments,old_transformed_segments
0,2021-12,38,0.057598,0.0,C,0.017192,2-medio,B
1,2021-12,50,0.069909,0.0,D,0.066528,3-alto,B
2,2021-12,63,0.058328,0.0,C,0.017192,2-medio,B
3,2021-12,81,0.208023,0.0,E,0.066528,3-alto,C
4,2021-12,95,0.074017,0.0,D,0.066528,3-alto,B


In [5]:
info_contracts = (
    pd.read_csv("data/info_contracts.csv")[["dt_ativacao", "id_imobiliaria", "id_contrato"]]
    .rename(columns={"id_contrato": "contract_id"})
)

info_contracts["dt_ativacao"] = pd.to_datetime(info_contracts["dt_ativacao"]).dt.strftime("%Y-%m")

In [166]:
df_bins.groupby("optimal_segments")["id_imobiliaria"].size()

optimal_segments
1-baixo      687
2-medio     1047
3-alto       481
4-novas    14091
Name: id_imobiliaria, dtype: int64

In [165]:
df = (
    info_contracts
    .merge(df_bins, on=["dt_ativacao", "id_imobiliaria"])
    .merge(scores_4KST, on=["dt_ativacao", "contract_id"], how="left")
)
df.head()

,dt_ativacao,id_imobiliaria,contract_id,predictions,target,old_segments,optimal_bins,optimal_segments,old_transformed_segments,rating_4KST
0,2021-12,30,441695,NaN,NaN,NaN,NaN,4-novas,NaN,B
1,2021-12,30,453263,NaN,NaN,NaN,NaN,4-novas,NaN,B
2,2022-01,30,480160,NaN,NaN,NaN,NaN,4-novas,NaN,B
3,2022-02,30,501215,NaN,NaN,NaN,NaN,4-novas,NaN,B
4,2022-02,30,503926,NaN,NaN,NaN,NaN,4-novas,NaN,C


#### Risco atual

In [19]:
calendar = pd.DataFrame(pd.date_range("2022-01", "2022-07", freq="M").strftime("%Y-%m"), columns=["dt_ativacao"])

In [20]:
old_score = pd.read_csv("data/old_risco_imob.csv")[["id_imobiliaria", "risco_imobiliaria"]].query("risco_imobiliaria != 0")

old_score["segments"] = (
    old_score["risco_imobiliaria"]
    .transform(
        lambda x: "A" if x == 2      else
                  "B" if x in (3, 4, 5, 6) else 
                  "C" if x == 7      else 
                  None
    )
)

old_score = old_score.merge(calendar, how="cross")
old_score.head()

,id_imobiliaria,risco_imobiliaria,segments,dt_ativacao
0,30,6,B,2022-01
1,30,6,B,2022-02
2,30,6,B,2022-03
3,30,6,B,2022-04
4,30,6,B,2022-05


In [21]:
old_score.groupby("segments")["id_imobiliaria"].size()

segments
A     288
B    2928
C    2154
Name: id_imobiliaria, dtype: int64

In [22]:
old_risk = info_contracts.merge(old_score[["dt_ativacao", "id_imobiliaria", "segments"]], on=["dt_ativacao", "id_imobiliaria"])
old_risk.head()

,dt_ativacao,id_imobiliaria,contract_id,segments
0,2022-01,30,480160,B
1,2022-02,30,501215,B
2,2022-02,30,503926,B
3,2022-02,30,511972,B
4,2022-02,30,534486,B


#### Calculadora

In [8]:
contracts = pd.read_parquet('../risk_suite/data/contracts.parquet')
defaults = pd.read_parquet('../risk_suite/data/defaults.parquet')
recoveries = pd.read_parquet('../risk_suite/data/recoveries.parquet')
revenues = pd.read_parquet('../risk_suite/data/revenues.parquet')

base_features = [
    'contract_id',
    'activation_date',
    'churn_date',
    'activation_month',
    'activation_quarter',
    'score_serasa',
    'rating',
    'rental_value',
]

contracts = contracts[base_features]
contracts = contracts.dropna(subset=['activation_date', 'rating'])
contracts = contracts.loc[lambda x: x['activation_quarter'] >= pd.Period('2020Q1')]
contracts = contracts.replace(['E1', 'E2', 'E3'], 'E')

## Auxiliary Functions

In [9]:
def report_economics2(aggkeys, revenues_pivot, defaults_pivot, recoveries_pivot):
    
    economics_df = pd.DataFrame(
        {
            'n_contracts': revenues_pivot.groupby(aggkeys).size(),
            'revenue_value': (revenues_pivot.sum(axis=1)).groupby(aggkeys).mean(),
            'prob_default': (defaults_pivot.sum(axis=1) > 0).groupby(aggkeys).mean(),
            'default_value': (defaults_pivot.sum(axis=1)).groupby(aggkeys).mean(),
            'recovery_value': (recoveries_pivot.sum(axis=1)).groupby(aggkeys).mean()
        }
    )

    economics_df = (
        economics_df
        .assign(recovery_efficiency=lambda x: x['recovery_value'] / x['default_value'])
        .assign(unit_economics=lambda x: x['revenue_value'] - x['default_value'] + x['recovery_value'])
        .assign(aggregate_margin=lambda x: x['unit_economics'] * x['n_contracts'])
    )

    return economics_df.sort_index(ascending=[True, False])

## Economics report

In [167]:
df = contracts.merge(df, on="contract_id")

In [168]:
econ_calculator = EconomicsCalculator(df, defaults, recoveries, revenues, max_history_date='2023-01')

defaults_pivot = econ_calculator._build_defaults_pivot()
recoveries_pivot = econ_calculator._build_recoveries_pivot()
revenues_pivot = econ_calculator._build_revenues_pivot()

/Users/raquel.camara/opt/anaconda3/lib/python3.9/site-packages/risk_suite/economics.py:139: FutureWarning: pivot_table dropped a column because it failed to aggregate. This behavior is deprecated and will raise in a future version of pandas. Select only the columns that can be aggregated.
  event_data


#### Optimal Segmentation

In [173]:
report_optimal = report_economics2(["optimal_segments"], revenues_pivot, defaults_pivot, recoveries_pivot)
report_optimal

,n_contracts,revenue_value,prob_default,default_value,recovery_value,recovery_efficiency,unit_economics,aggregate_margin
optimal_segments,,,,,,,,
1-baixo,4179,1279.719186,0.070830,438.531548,125.268579,0.285655,966.456217,4038820.53
2-medio,12949,1204.807049,0.113136,780.734061,198.346390,0.254051,622.419378,8059708.52
3-alto,17751,1222.364521,0.145006,1046.547938,271.636124,0.259554,447.452707,7942733.00
4-novas,36726,1266.555032,0.101209,665.675988,166.292780,0.249810,767.171824,28175152.40


In [169]:
report_opt_rating = report_economics2(["optimal_segments", "rating_4KST"], revenues_pivot, defaults_pivot, recoveries_pivot)

In [172]:
pd.pivot_table(report_opt_rating, values="unit_economics", index="optimal_segments", columns="rating_4KST")

rating_4KST,A,B,C,D,E,N/I
optimal_segments,,,,,,
1-baixo,1278.798329,1267.063339,829.785789,568.691250,335.305081,1796.533333
2-medio,1308.235244,1037.317666,720.226565,328.677996,-250.968656,1770.756667
3-alto,1319.298685,1162.404541,633.574484,103.102561,-735.995753,2507.200000
4-novas,1457.137579,1242.488676,856.024570,485.503035,-108.714041,3117.733333


In [37]:
# Unit economics por segmento/contrato
pd.pivot_table(report_opt_rating, values="unit_economics", index="optimal_segments", columns="rating_4KST")

rating_4KST,A,B,C,D,E
optimal_segments,,,,,
1-baixo,1963.697122,1641.155619,1111.575175,241.329491,-609.575301
2-medio,1903.816954,1331.554832,650.290520,41.085374,-1298.277915
3-alto,1746.346473,1237.654170,448.854299,-774.045603,-2285.534925


In [38]:
# Probabilidade de default por segmento/contrato
pd.pivot_table(report_opt_rating, values="prob_default", index="optimal_segments", columns="rating_4KST")

rating_4KST,A,B,C,D,E
optimal_segments,,,,,
1-baixo,0.037829,0.074273,0.134340,0.203523,0.289157
2-medio,0.042146,0.093693,0.162638,0.253995,0.372731
3-alto,0.052506,0.111219,0.198349,0.295235,0.449050


In [41]:
# Eficiência de Cobrança por segmento/contrato
pd.pivot_table(report_opt_rating, values="recovery_efficiency", index="optimal_segments", columns="rating_4KST")

rating_4KST,A,B,C,D,E
optimal_segments,,,,,
1-baixo,0.427047,0.415576,0.364942,0.184727,0.176162
2-medio,0.417388,0.328387,0.294109,0.252898,0.182399
3-alto,0.437537,0.355904,0.308290,0.202930,0.158429


In [42]:
# Mix de contratos por segmento/contrato
pd.pivot_table(report_opt_rating, values="n_contracts", index="optimal_segments", columns="rating_4KST")

rating_4KST,A,B,C,D,E
optimal_segments,,,,,
1-baixo,608,929,1258,511,830
2-medio,1044,1633,2441,1189,2259
3-alto,2095,3084,5329,2686,6104


#### Old Segmentation

In [40]:
report_old = report_economics2(["old_segments"], revenues_pivot, defaults_pivot, recoveries_pivot)
report_old

,n_contracts,revenue_value,prob_default,default_value,recovery_value,recovery_efficiency,unit_economics,aggregate_margin
old_segments,,,,,,,,
A,1271,1335.677663,0.052714,270.501857,98.335980,0.363532,1163.511786,1478823.48
B,3354,1214.062275,0.064103,347.241855,107.349120,0.309148,974.169541,3267364.64
C,5833,1140.376235,0.065832,353.497725,107.667615,0.304578,894.546125,5217887.55
D,5967,1132.072073,0.108765,696.862427,195.004444,0.279832,630.214091,3760487.48
E,18454,1136.454054,0.124688,870.064657,239.138413,0.274851,505.527810,9329010.21


##### Transformed

In [41]:
report_old_transformed = report_economics2(["old_transformed_segments"], revenues_pivot, defaults_pivot, recoveries_pivot)
report_old_transformed

,n_contracts,revenue_value,prob_default,default_value,recovery_value,recovery_efficiency,unit_economics,aggregate_margin
old_transformed_segments,,,,,,,,
A,1271,1335.677663,0.052714,270.501857,98.335980,0.363532,1163.511786,1478823.48
B,15154,1153.415172,0.082354,487.315528,141.986648,0.291365,808.086292,12245739.67
C,18454,1136.454054,0.124688,870.064657,239.138413,0.274851,505.527810,9329010.21


#### Risco Atual

In [68]:
old_risk_2 = contracts.merge(old_risk, on="contract_id")

In [69]:
econ_calculator_old = EconomicsCalculator(old_risk_2, defaults, recoveries, revenues, max_history_date='2023-01')

defaults_pivot_old = econ_calculator_old._build_defaults_pivot()
recoveries_pivot_old = econ_calculator_old._build_recoveries_pivot()
revenues_pivot_old = econ_calculator_old._build_revenues_pivot()

/Users/raquel.camara/opt/anaconda3/lib/python3.9/site-packages/risk_suite/economics.py:139: FutureWarning: pivot_table dropped a column because it failed to aggregate. This behavior is deprecated and will raise in a future version of pandas. Select only the columns that can be aggregated.
  event_data


In [84]:
report_old_rating = report_economics2(["segments", "rating"], revenues_pivot_old, defaults_pivot_old, recoveries_pivot_old)

report_old_risk = report_economics2(["segments"], revenues_pivot_old, defaults_pivot_old, recoveries_pivot_old)
report_old_risk

,n_contracts,revenue_value,prob_default,default_value,recovery_value,recovery_efficiency,unit_economics,aggregate_margin
segments,,,,,,,,
A,1570,1137.982134,0.026752,164.519268,28.650713,0.174148,1002.113580,1573318.32
B,24365,1191.220415,0.071086,454.496445,143.236561,0.315154,879.960531,21440238.33
C,22243,1124.420053,0.136762,875.796947,236.966910,0.270573,485.590016,10800978.73


In [86]:
# Probabilidade de default por segmento/contrato
pd.pivot_table(report_old_rating, values="n_contracts", index="segments", columns="rating")

rating,A,B,C,D,E
segments,,,,,
A,308,356,492,199,215
B,3956,5522,8168,3017,3702
C,3253,5181,7704,2647,3458
